# Import

In [1]:
! pip install category_encoders

You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
! pip install eli5

You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import model_preprocessing as mp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import category_encoders
from category_encoders.one_hot import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import eli5

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [4]:
df = pd.read_csv('verlander.csv')
df.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (131) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat,overall_FF_perc,count_cat_FF_perc,overall_SL_perc,count_cat_SL_perc,overall_CU_perc,count_cat_CU_perc,overall_CH_perc,count_cat_CH_perc,overall_FC_perc,count_cat_FC_perc,overall_FT_perc,count_cat_FT_perc,pitch_count,L1_pitch_type,L1_pitch_result,L1_pitch_zone,L5_FF_perc,L5_strike_perc,L5_SL_perc,L5_CU_perc,L5_CH_perc,L5_FC_perc,batting_order_slot,pitcher_AB
0,1510,FF,2018-03-29,22730,94.6,-1.9486,6.8034,592261.0,434378.0,8.0,NaN,2018.0,-0.8339,1.6529,0.2748,1.7070,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,7.4453,-136.8952,-10.8637,-11.8856,32.3168,-9.8095,3.3339,1.4562,NaN,NaN,NaN,92.490,2652.0,5.498,529411.0,55.0008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,S,NaN,Bot,180329_195119,Standard,1.0,NaN,False,False,False,False,0,0,1,0,66.241684,21.138212,42.05021,40.0,0.300159,0.220395,0.098684,26.995565,27.516779,36.70213,43.915344,0.231478,0.2,0.054545,6.76275,36.231884,21.666666,33.962265,0.426231,0.25,0.041667,0.0,0,0,Justin Verlander,NaN,called_strike,NaN,4-Seam Fastball,Standard,0,neutral,fastball,0.583272,0.685385,0.215498,0.163427,0.151046,0.114471,0.039606,0.029518,0.007626,0.005040,0.002952,0.002160,1,FF,first pitch,-1.0,56.076277,54.250162,24.676794,16.305753,2.60181,0.339367,1,False
1,1511,FF,2018-03-29,22711,94.3,-1.9916,6.6522,592261.0,434378.0,14.0,NaN,2018.0,-1.2892,1.4338,0.4047,1.2020,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,8.9916,-136.5666,-11.4425,-18.0155,31.9642,-12.0415,3.3446,1.4668,NaN,NaN,NaN,93.284,2638.0,6.095,529411.0,54.4034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,B,NaN,Bot,180329_195137,Standard,1.0,NaN,False,True,False,False,0,0,0,0,66.241684,21.138212,42.05021,40.0,0.300159,0.220395,0.098684,26.995565,27.516779,36.70213,43.915344,0.231478,0.2,0.054545,6.76275,36.231884,21.666666,33.962265,0.426231,0.25,0.041667,0.0,0,1,Justin Verlander,NaN,ball,NaN,4-Seam Fastball,Standard,1,ahead,fastball,0.583272,0.493105,0.215498,0.257727,0.151046,0.198288,0.039606,0.039467,0.007626,0.008084,0.002952,0.003329,2,FF,S,8.0,56.076277,54.250162,24.676794,16.305753,2.60181,0.339367,1,False
2,1512,SL,2018-03-29,22699,86.4,-2.1580,6.5734,592261.0,434378.0,14.0,NaN,2018.0,0.2707,0.3526,1.5138,-0.0429,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,8.0583,-125.1452,-10.0136,1.1868,24.5998,-26.8457,3.3339,1.4562,NaN,NaN,NaN,84.944,2680.0,5.543,529411.0,54.9560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,B,NaN,Bot,180329_195201,Standard,1.0,NaN,False,True,False,True,0,0,0,0,66.241684,21.138212,42.05021,40.0,0.300159,0.220395,0.098684,26.995565,27.516779,36.70213,43.915344,0.231478,0.2,0

# Clean

In [5]:
processor = mp.Preprocess(dataframe = df)
df = processor.process(processor.df)
df.head()

# Create Feature Matrix and Target Vector

In [8]:
feature_matrix = df.drop(columns = ['next_pitch'])
target_vector = df['next_pitch']

feature_matrix.shape, target_vector.shape

((6188, 328), (6188,))

# Train Test Split

In [9]:
x_train, x_test, y_train, y_test = train_test_split(feature_matrix, target_vector)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4641, 328), (1547, 328), (4641,), (1547,))

# Instantiate Model

[LogisticRegression Docs](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [26]:
clf = Pipeline(steps = [('scaler', RobustScaler()),
                        ('log', LogisticRegression(
                            #penalty = 'l1',
                            #C = 1,
                            #multi_class = 'ovr'
                        ))])


params = {
    #'log__intercept_scaling': [1, 10, 100, 1000],
    #'log__tol': [.001, .0001, .01],
    'log__solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'log__max_iter': [1000]
}


grid = GridSearchCV(
    estimator = clf,
    scoring = 'accuracy',
    param_grid = params,
    refit = True,
    cv = 10,
    verbose = 10,
    n_jobs = -1)

# Train Model

In [27]:
grid.fit(x_train, y_train)

Fitting 10 folds for each of 15 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   19.7s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:   27.1s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 135 out of 150 | elapsed: 11.8min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 15.6min finished
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silen

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('scaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('log', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'log__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'log__max_iter': [1000, 10000, 100000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=10)

In [29]:
pd.DataFrame(grid.cv_results_).sort_values(by = 'mean_test_score', ascending = False).head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, *

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_log__max_iter,param_log__solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
3,107.129369,0.810546,0.028457,0.000898,1000,sag,"{'log__max_iter': 1000, 'log__solver': 'sag'}",0.598712,0.603004,0.583691,0.563441,0.55819,0.593952,0.596112,0.609071,0.598272,0.571429,0.587589,0.016654,1,0.639281,0.639521,0.638323,0.641523,0.645679,0.642891,0.637147,0.637626,0.636668,0.639148,0.639781,0.002676
4,130.190367,0.761600,0.029062,0.001357,1000,saga,"{'log__max_iter': 1000, 'log__solver': 'saga'}",0.598712,0.598712,0.585837,0.569892,0.55819,0.589633,0.591793,0.609071,0.598272,0.573593,0.587373,0.014900,2,0.636886,0.636168,0.636886,0.639368,0.645439,0.641934,0.634035,0.633317,0.633317,0.636755,0.637411,0.003690
14,235.332427,40.323523,0.017609,0.000218,100000,saga,"{'log__max_iter': 100000, 'log__solver': 'saga'}",0.596567,0.598712,0.583691,0.561290,0.55819,0.593952,0.596112,0.606911,0.600432,0.569264,0.586512,0.016583,3,0.639760,0.640719,0.638563,0.641523,0.645918,0.643849,0.638583,0.638344,0.637147,0.639148,0.640355,0.002591
9,330.093505,22.179186,0.027958,0.000334,10000,saga,"{'log__max_iter': 10000, 'log__solver': 'saga'}",0.596567,0.598712,0.583691,0.561290,0.55819,0.593952,0.596112,0.606911,0.598272,0.569264,0.586296,0.016414,4,0.639760,0.640719,0.638563,0.641523,0.645918,0.643849,0.638583,0.638344,0.637147,0.639148,0.640355,0.002591
2,2.445486,0.162159,0.031205,0.002910,1000,liblinear,"{'log__max_iter': 1000, 'log__solver': 'liblin...",0.594421,0.594421,0.581545,0.561290,0.55819,0.593952,0.596112,0.606911,0.600432,0.564935,0.585219,0.016710,5,0.640719,0.641198,0.639760,0.641762,0.646876,0.645045,0.640737,0.639780,0.640019,0.641302,0.641720,0.002248


# Evaluate Model

In [30]:
model = grid.best_estimator_
model

Pipeline(memory=None,
     steps=[('scaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('log', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='sag',
          tol=0.0001, verbose=0, warm_start=False))])

In [31]:
accuracy_score(model.predict(x_train), y_train)

0.6365007541478129

In [32]:
df['next_pitch'].value_counts(normalize = True)

FF    0.560763
SL    0.246768
CU    0.163058
CH    0.026018
FC    0.003394
Name: next_pitch, dtype: float64

In [33]:
eli5.show_weights(model, feature_names = feature_matrix.columns)

In [34]:
accuracy_score(model.predict(x_test), y_test)

0.5966386554621849